# Demo notebook for Two Model Readers
For each model reader, the user is shown how to find the desired variables across model results, how to determine what the correct file input is for the model's reader, and how to run the reader. Note the syntax for each is identical. Any differences in inputs are easily accessible to the user through the methods shown.

In [ ]:
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Choose_Model('')  # Show list of models currently available

In [ ]:
# Find the desired variable
help(MW.Variable_Search)

In [ ]:
# Find what model has the variables...
MW.Variable_Search('temperature')

In [ ]:
# then add the model name and file_dir to find which files have the variables.
# Usually of the files for a given model run have the same variables.
ctipe_file_dir = 'C:/Users/rringuet/Kamodo_Data/CTIPe/Storm_201303/'
MW.Variable_Search('temperature', model='CTIPe', file_dir=ctipe_file_dir)

In [ ]:
# Find correct input file pattern for the reader
ctipe_files = MW.FileSearch('CTIPe', ctipe_file_dir)
ctipe_files

In [ ]:
# Retrieve model reader and access model data for the CTIPe model.
# Note the variable chosen depends on pressure level, but the user
# does not need to know that to be able to retrieve and interact with it.
reader = MW.Model_Reader('CTIPe')
kamodo_object_ctipe = reader(ctipe_files[1],
                             variables_requested=['T_n'])
kamodo_object_ctipe

In [ ]:
# Find related variables in the GITM data
gitm_file_dir = 'C:/Users/rringuet/Kamodo_Data/GITM/Storm_201303/'
MW.Variable_Search('temperature', model='GITM', file_dir=gitm_file_dir)

In [ ]:
# Find correct file pattern for the reader
gitm_file_dir = 'C:/Users/rringuet/Kamodo_Data/GITM/Storm_201303/'
gitm_files = MW.FileSearch('GITM', gitm_file_dir)
gitm_files

In [ ]:
# Retrieve model reader and access model data for the GITM model.
# Note the identical syntax!
reader = MW.Model_Reader('GITM')
kamodo_object_gitm = reader(gitm_files[0],
                            variables_requested=['T_n'])
kamodo_object_gitm

In [ ]:
# Collect functions into a single Kamodo object
from kamodo import Kamodo
kamodo_object = Kamodo()
kamodo_object['CTIPeT_n[K]'] = kamodo_object_ctipe['T_n_ijk']
kamodo_object['GITMT_n[K]'] = kamodo_object_gitm['T_n_ijk']
kamodo_object

In [ ]:
# Find the range for each coordinate for the ctipe function to aid in plotting choices.
# Note that this only works on gridded functions (e.g. '_ijk' functions).
from kamodo import get_defaults
ctipe_defaults = get_defaults(kamodo_object.CTIPeT_n)
for key in ctipe_defaults:
    print(key, ctipe_defaults[key].min(), ctipe_defaults[key].max())

In [ ]:
# Repeat to find the range for each coordinate for the gitm function.
# Note the difference in height ranges.
gitm_defaults = get_defaults(kamodo_object.GITMT_n)
for key in gitm_defaults:
    print(key, gitm_defaults[key].min(), gitm_defaults[key].max())

In [ ]:
# Plot each. Note that the user can automatically slice a pressure level variable by height!
# Areas of the grid where the height is greater than the pressure level will be blank.
fig = kamodo_object.plot('CTIPeT_n', plot_partial={'CTIPeT_n': {'time': 12., 'height': 350.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
fig = kamodo_object.plot('GITMT_n', plot_partial={'GITMT_n': {'time': 12., 'height': 350.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Example of a default Kamodo plot
kamodo_object.plot('GITMT_n', plot_partial={'GITMT_n': {'time': 12., 'height': 350.}})

In [ ]:
# Directly compare corresponding slices.
kamodo_object.plot('CTIPeT_n', 'GITMT_n',
                   plot_partial={'CTIPeT_n': {'time': 12., 'lat': 25., 'height': 350.},
                                 'GITMT_n': {'time': 12., 'lat': 25., 'height': 350.}})

In [ ]:
# Demostrate syntax to access the regular interpolator
# Best for single point calculations
kamodo_object_ctipe.T_n([12, -20, 20, 350])

In [ ]:
# Demostrate syntax to access the gridded interpolator
# Best for slice calculations
kamodo_object_ctipe.T_n_ijk(time=12, lon=-20, lat=20, height=350)

In [ ]:
# At what pressure level does a given height correspond to in the CTIPe data?
# The answer depends on the time, longitude, and latitude.
kamodo_object_ctipe.Plev_ijk(time=12, lon=-20, lat=20, height=350)

In [ ]:
# How does the corresponding pressure level vary at a given latitude and time?
kamodo_object_ctipe.plot('Plev_ijk', plot_partial={'Plev_ijk': {'time': 12., 'lat': 20.,
                                                                'height': 350.}})